In [1]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
# !kill -9 -1

!apt-get update -qq 2>&1 > /dev/null
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse



gpg: keybox '/tmp/tmpm709544v/pubring.gpg' created
gpg: /tmp/tmpm709544v/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [2]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

# # Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
# Work around misordering of STREAM and STDIN in Jupyter.
# https://github.com/jupyter/notebook/issues/3159
prompt = !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
# prompt
vcode = getpass.getpass(prompt[0] + '\n\nEnter verification code: ')
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

IndexError: ignored

In [3]:
# Create a directory and mount Google Drive using that directory.
!mkdir -p new
!google-drive-ocamlfuse new

print ('Files in Drive:')
!ls new/ #Year\ 3:\ Focus/Summer Internships/Documents

# # Create a file in Drive.
# !echo "This newly created file will appear in your Drive file list." > drive/created.txt

import numpy as np
# X = np.load("new/pred.npy")
# Y_ = np.load("new/target.npy")



fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option
Files in Drive:
000009045073-101010016272.pdf
000009045073-105010097054.pdf
101009888218.pdf
101010016272.pdf
105009454758.pdf
105009585929.pdf
105010097054.pdf
118009434025.pdf
1.2 Make-up Work - Huy Nguyen.pdf
1.2 Make-up Work.pdf
2014  Notice of assessment - My Account.pdf
3.2 Pre-class Work.odt
337 Tranquille Road, Kamloops - Financial Statement.pdf
4.1 Make up work - Huy Nguyen.pdf
4.1 Make up work.pdf
541590XXXXXX5646-2016Aug16-2016Sep13.pdf
541590XXXXXX5646-2016Jul14-2016Aug15.pdf
541590XXXXXX5646-2016Jun14-2016Jul13.pdf
541590XXXXXX5646-2016Nov15-2016Dec13.pdf
541590XXXXXX5646-2016Oct14-2016Nov14.pdf
541590XXXXXX5646-2016Sep14-2016Oct13.pdf
541590XXXXXX5646-2017Apr14-2017May15.pdf
541590XXXXXX5646-2017Feb14-2017Mar13.pdf
541590XXXXXX5646-2017Jul14-2017Aug14.pdf
541590XXXXXX5646-2017Jun14-2017Jul13.pdf
541590XXXXXX5646-2017Mar14-2017Apr13.pdf
541590XXXXXX5646-2017Nov15-2017Dec13.pdf

In [4]:
import numpy as np
!ls "new/Year 3: Focus/Summer Internships/Documents"
X = np.load("new/Year 3: Focus/Summer Internships/Documents/pred.npy")
Y_ = np.load("new/Year 3: Focus/Summer Internships/Documents/target.npy")

1D_Diagonalize_Recursion.ipynb
384_111_Leela_K80_v2_1.ipynb
384_111_Leela_K80_v2.ipynb
Compare_diagonalize_recursion.ipynb
Copy of Untitled0.ipynb
diagonalize
diagonalize_3D_Hamiltonian.py
Diagonalize_Recursion.ipynb
eigenvalue.npy
eigenvector.npy
greenf.py
H.py
prediction.npy
pred.npy
target.npy
test1.py
test.py
true.npy
Untitled0.ipynb
Untitled1.ipynb
Untitled.ipynb
weights-improvement-1-1.0178566742420567.hdf5
weights-improvement-2-1.0170094634986524.hdf5


In [5]:
np.std(Y_)

1.0000000000000004

In [0]:

from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, Y_, test_size=0.2, random_state=42)
# print (X_train.shape)
from keras.models import Sequential, load_model
from keras.layers import Dense, BatchNormalization
from keras.wrappers.scikit_learn import KerasRegressor
from keras import regularizers
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from keras.callbacks import ModelCheckpoint


seed = 7
np.random.seed(seed)




# create model
model = Sequential()
model.add(Dense(200, input_dim=10, kernel_initializer='normal', activation='relu', kernel_regularizer=regularizers.l2(0.0001)))
model.add(BatchNormalization())
model.add(Dense(200, input_dim=200, kernel_initializer='normal', activation='relu', kernel_regularizer=regularizers.l2(0.0001)))
model.add(BatchNormalization())
model.add(Dense(400, input_dim=200, kernel_initializer='normal', activation='relu', kernel_regularizer=regularizers.l2(0.0001)))
model.add(BatchNormalization())
model.add(Dense(800, input_dim=400, kernel_initializer='normal', activation='relu', kernel_regularizer=regularizers.l2(0.0001)))
model.add(BatchNormalization())
model.add(Dense(400, input_dim=800, kernel_initializer='normal', activation='relu', kernel_regularizer=regularizers.l2(0.0001)))
model.add(BatchNormalization())
model.add(Dense(100, kernel_initializer='normal', activation='relu', kernel_regularizer=regularizers.l2(0.0001)))
model.add(BatchNormalization())
model.add(Dense(10, kernel_initializer='normal', activation='relu', kernel_regularizer=regularizers.l2(0.0001)))
model.add(BatchNormalization())
model.add(Dense(1, kernel_initializer='normal', kernel_regularizer=regularizers.l2(0.0001)))
print(model.summary())

# Compile model
model.compile(loss='mean_squared_error', optimizer='adam', metrics = ["mean_squared_error"])

filepath="new/Year 3: Focus/Summer Internships/Documents/weights-improvement-{epoch}-{mean_squared_error}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='mean_squared_error', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]




# kfold = KFold(n_splits=2, random_state=seed)
# results = cross_val_score(estimator, X_train, y_train, cv=kfold)
# print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))
# print ("Results", results)
model = load_model('new/Year 3: Focus/Summer Internships/Documents/weights-improvement-2-1.0170094634986524.hdf5')
model.fit(X, Y_, validation_split=0.2, epochs=150, batch_size=50, callbacks=callbacks_list, verbose=1)

model.save('new/Year 3: Focus/Summer Internships/Documents/my_model.hdf5')

# from google.colab import files
# files.download( 'new/my_model.h5' ) 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 200)               2200      
_________________________________________________________________
batch_normalization_8 (Batch (None, 200)               800       
_________________________________________________________________
dense_10 (Dense)             (None, 200)               40200     
_________________________________________________________________
batch_normalization_9 (Batch (None, 200)               800       
_________________________________________________________________
dense_11 (Dense)             (None, 400)               80400     
_________________________________________________________________
batch_normalization_10 (Batc (None, 400)               1600      
_________________________________________________________________
dense_12 (Dense)             (None, 800)               320800    
__________

  48200/3283692 [..............................] - ETA: 1:26:50 - loss: 1.6547 - mean_squared_error: 1.6532

  92700/3283692 [..............................] - ETA: 1:21:42 - loss: 5.5972 - mean_squared_error: 5.5956

 141100/3283692 [>.............................] - ETA: 1:17:26 - loss: 3.9470 - mean_squared_error: 3.9453

 185650/3283692 [>.............................] - ETA: 1:15:26 - loss: 3.1605 - mean_squared_error: 3.1590

...

1935100/3283692 [================>.............] - ETA: 33:16 - loss: 1.1844 - mean_squared_error: 1.1835

1981350/3283692 [=================>............] - ETA: 32:06 - loss: 1.1694 - mean_squared_error: 1.1686

2028150/3283692 [=================>............] - ETA: 30:58 - loss: 1.1581 - mean_squared_error: 1.1573

2074650/3283692 [=================>............] - ETA: 29:50 - loss: 1.1443 - mean_squared_error: 1.1434

2119850/3283692 [==================>...........] - ETA: 28:42 - loss: 1.1271 - mean_squared_error: 1.1262